In [167]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import PillowWriter
from matplotlib import _cm
from skimage import color
from skimage import io
import numba
from numba import jit



In [168]:
edge = np.linspace(0, 1, 1000) #100 points from 0 to 1 including 0 and 1
xv, yv = np.meshgrid(edge, edge)

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# 1. Load the image
img = Image.open('turkey.png')

img = img.resize((100, 100))
plt.imshow(img)
img_np = np.array(img)
if img_np.shape[-1] == 4:
    img_np = img_np[:, :, :3]  # Keep only RGB
    
img_norm = img_np / 255.0
img_gray = np.mean(img_norm, axis=2)  # (height, width)

# # 7. Threshold to get binary turkey mask
turkey_bool = img_gray < 0.9  # adjust threshold slightly if needed!
turkey_bool = np.flip(turkey_bool, axis=0)




In [ ]:
turkey_bool

In [155]:
a_turk = 1.32e-7

In [156]:
rawturk_temp = 275.15 + 25
oven_temp = 275.15 + 165

In [157]:
init_heat = np.zeros([100, 100]) + oven_temp
init_heat[turkey_bool] = rawturk_temp

In [ ]:
plt.contourf(init_heat)

In [159]:
times = 36000
times_snapshot = 3600
f = int(times/times_snapshot)
heat_frames = np.zeros([times_snapshot, 100, 100])
heat_frames[0] = init_heat


In [160]:
x = 0.5
dx = 0.5/100
dt = 1

In [ ]:
a_turk*dt/dx**2

In [162]:
@numba.jit("f8[:,:,:](f8[:,:,:],b1[:,:])", nopython=True, nogil=True)
def solve_heat(heatmap, is_turkey):
    cs = heatmap[0].copy() #current state
    length = len(cs[0])
    cf = 0 #current frame
    for t in range(1, times):
        ns = cs.copy()
        for i in range(1, length-1):
            for j in range(1, length -1):
                if is_turkey[j][i]:
                    a = a_turk
                    ns[j][i] = cs[j][i] +a*dt/dx**2 * (cs[j+1][i] + cs[j-1][i] +\
                                                       cs[j][i+1] + cs[j][i-1] -\
                                                        4*cs[j][i])
        cs = ns.copy()
        if t%f ==0:
            cf = cf +1
            heatmap[cf] = cs
    return heatmap

In [163]:
heat_frames = solve_heat(heat_frames, turkey_bool)

In [164]:
heat_frames -=273.15

In [ ]:
my_cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
a = plt.contourf(heat_frames[900], 100, cmap = my_cmap, vmin=rawturk_temp - 273.15, vmax=oven_temp-273.15)
plt.colorbar(a)

In [ ]:
def animate(i):
    ax.clear()
    ax.contourf(heat_frames[10*i], 100, cmap = my_cmap, vmin = rawturk_temp-273.15, vmax = oven_temp-273.15)

    return fig,
fig, ax = plt.subplots(figsize=(8,6))
ani = animation.FuncAnimation(fig, animate, frames = 359, interval=50)
ani.save('cooked.gif', writer='pillow', fps=20)